In [18]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [19]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '49b66289c4a6617ff506de0ecd0e7a34', 'Date': 'Wed, 28 Aug 2024 02:27:06 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [20]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Chen',
  'subject': 'Biology',
  'stars': 5,
  'review': "Dr. Chen's lectures are engaging and her lab sessions are incredibly informative. Best professor I've had!"},
 {'professor': 'Prof. Michael Brown',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Brown brings history to life with his storytelling. Sometimes goes off-topic, but always interesting.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Dr. Johnson knows her stuff, but can be a bit dry in lectures. Office hours are helpful.'},
 {'professor': 'Prof. David Lee',
  'subject': 'Computer Science',
  'stars': 5,
  'review': "Prof. Lee's practical approach to teaching programming is excellent. Very patient with beginners."},
 {'professor': 'Dr. Rachel Green',
  'subject': 'Psychology',
  'stars': 4,
  'review': "Dr. Green's research experience adds depth to her lectures. Assignments can be challenging but rewarding."},
 {'professor': 'Prof. Thomas Wilso

In [21]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [22]:
processed_data[0]

{'values': [-0.012916025,
  -0.03919934,
  -0.01576999,
  0.041199557,
  0.021319369,
  0.013501453,
  0.011299997,
  0.051810455,
  -0.050542027,
  0.0032625464,
  0.01775801,
  -0.0058756326,
  0.0007485562,
  -0.005189583,
  0.014721097,
  0.02350253,
  -0.010177925,
  -0.018623956,
  0.019038634,
  0.036833234,
  0.030052016,
  -0.026637014,
  0.04393156,
  -0.031564374,
  -0.043468095,
  -0.084838405,
  0.020453421,
  -0.006064677,
  0.02073394,
  0.029027516,
  0.05210317,
  -0.01639201,
  -0.018123902,
  -0.023770852,
  -0.04146788,
  0.071861394,
  -0.023758655,
  0.028173765,
  0.005637802,
  -0.01487965,
  0.05464003,
  0.03188148,
  -0.01487965,
  0.025905227,
  0.028124979,
  -0.015379705,
  -0.037808947,
  -0.0059671057,
  0.031661946,
  0.02022169,
  -0.05259103,
  0.024966102,
  0.029588552,
  -0.0044181584,
  -0.06273846,
  0.022929298,
  0.028881159,
  0.046053737,
  0.0041406895,
  -0.0366137,
  0.07000754,
  0.003997382,
  0.014525954,
  -0.009531514,
  -0.03610145,
